In [1]:
%%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});

<IPython.core.display.Javascript object>

In [2]:
%%javascript
MathJax.Hub.Queue(
  ["resetEquationNumbers", MathJax.InputJax.TeX],
  ["PreProcess", MathJax.Hub],
  ["Reprocess", MathJax.Hub]
);

<IPython.core.display.Javascript object>

# <ins>Reflection</ins>

This project aimed to explore the use of parallel computing when training a convolutional neural network (CNN) for image classification. Specifically, I wanted to understand how using the parallel computation processes of graphics processing units (GPUs) can increase the training speed of more complicated and deeper convolutional neural networks. Naturally, the use of these more sophisticated models necessitated the understanding of many more concepts than those of a simple CNN. After this model was deemed to fall short of the desired performance, we explored multiple new implementations. This introduction of more complex models led to the exploration of various concepts used in the optimisation of more advanced models; this provided a great framework and path for the progression of the project. Other goals of the project were to explore the features of the deep learning framework of PyTorch, and examine the deep learning decisions made to improve the models. Furthermore, I wanted to attempt to provide a reasonable explanation of how CNNs work, and explain some of the mathematical underpinnings of various concepts, such as cross-entropy loss, stochastic gradient descent, batch normalisation, and residual blocks. 

Having chosen the field of image classification for the project, I was able to discover that the CIFAR-10 dataset is a very popular choice in this field. From there, I discovered that convolutional neural networks were the main way to implement models that classify images. 

The programming was completed entirely in Python, with a brief exploration of Compute Unified Device Architecture (CUDA); however, this proved hard to run in a Jupyter Notebook file. Had I more time, I should have liked to explore code that used CUDA directly in more depth. CUDA is designed to work with languages such as C++ and C. These are programming languages that I am not very familiar with, and due to the time constraints of the project, I did not explore them further. Moreover, there are many resources that make Python the more tractable programming language in this field by making use of frameworks such as PyTorch and TensorFlow. Many resources used in this project have been cited, including various sections of code, websites, and links to papers. Every code resource had to be understood, and adapted for the needs of the project. In the literature, I read about the use of ResNet as a more advanced model, and I explained some of its concepts.


### <ins>The overall method of the project was as follows:</ins>

1. Find a suitable dataset.
2. Research about CNNs.
3. Implement a CNN.
3. Improve this classifier via parameter optimisation — this requires GPU to speed up computation time.
4. A look at how GPU computation fares against that of CPU for operations performed in CNNs.
5. Finally, implementing a model (ResNet) that uses more state-of-the-art concepts to reach a higher accuracy, while also utilising, and explaining, various optimisation concepts.
6. Conclusion and future work.

### <ins>Parallelisation paradigm explored</ins>

The parallelisation paradigm explored was GPU parallel computing. The main operation of convolution in the eponymous CNN is fundamental to its function. It involves the element-wise matrix multiplication of a small kernel matrix, with patches/regions (or blocks) of an input image – represented as a matrix. The kernel is shifted across the entire image matrix. These calculations are independent, and they can, therefore, be performed simultaneously in a parallel manner. As discussed in depth in the project, this is a computational process that directly aligns with the superior architecture of a GPU. 

Mathematically, if we have an $n \times n$ input matrix, $A$, and a $3 \times 3$ kernel matrix, $K$, the operation is represented in the following way.

$$
\begin{equation}
 A * K =
\begin{pmatrix}
  a_{11}  & a_{12}  & a_{13}  & \cdots  & a_{1n} \\
  a_{21}  & a_{22}  & a_{23}  & \cdots  & a_{2n} \\
  a_{31}  & a_{32}  & a_{33}  & \cdots  & a_{3n} \\
  \vdots  & \vdots  & \vdots  & \ddots  & \vdots \\
  a_{n1}  & a_{n2}  & a_{n3}  & \cdots  & a_{nn} \\
\end{pmatrix}
*
\begin{pmatrix}
  k_{11}  & k_{12}  & k_{13} \\
  k_{21}  & k_{22}  & k_{23} \\
  k_{31}  & k_{32}  & k_{33} \\
\end{pmatrix}
\end{equation}
$$

We can split $A$ into regions, and compute the operation in the following way. This is a concept known as block multiplication, and we saw it explained in the project. Here, $m = n-2$.

$$
\begin{equation}
 A * K =
\begin{pmatrix}
  A_{11}  & A_{12}  & \cdots  & A_{1m} \\
  A_{21}  & A_{22}  & \cdots  & A_{2m} \\
  \vdots  & \vdots  & \ddots  & \vdots \\
  A_{m1}  & A_{m2}  & \cdots  & A_{mm} \\
\end{pmatrix}
* K =
\begin{pmatrix}
  A_{11}*K  & A_{12}*K  & \cdots  & A_{1m}*K \\
  A_{21}*K  & A_{22}*K  & \cdots  & A_{2m}*K \\
  \vdots  & \vdots  & \ddots  & \vdots \\
  A_{m1}*K  & A_{m2}*K  & \cdots  & A_{mm}*K \\
\end{pmatrix}
\end{equation}
$$

Here, the regions $A_{ij}$ are defined by each small $3 \times 3$ matrix that is iterated over by the $3 \times 3$ kernel. For a single patch operation, we have that

$$
\begin{equation}
 A_{11} * K =
\begin{pmatrix}
  a_{11}  & a_{12}  & a_{13} \\
  a_{21}  & a_{22}  & a_{23} \\
  a_{31}  & a_{32}  & a_{33} \\
\end{pmatrix}
*
\begin{pmatrix}
  k_{11}  & k_{12}  & k_{13} \\
  k_{21}  & k_{22}  & k_{23} \\
  k_{31}  & k_{32}  & k_{33} \\
\end{pmatrix}
= \sum_{i=1}^{3} \sum_{y=1}^{3} a_{ij}k_{ij}.
\end{equation}
$$

The element-wise multiplication and addition operations in this process are computationally simple, and we can see that they are highly parallelisable, since each of the operations $A_{ij}*K$ are independent. GPUs have thousands of cores, therefore they can handle many such tasks simultaneously. This can greatly reduce the training time of CNNs. This is beneficial since operations on entire layers of a CNN can be computed more efficiently.

This form of block multiplication is more efficient, since it makes use of the shared memory inside streamline multiprocessors (SMs) within GPUs. SMs are the heart of a GPU. Multiple SMs are present within each GPU; they are individual processing units responsible for completing multiple tasks in parallel. SMs are comprised of arithmetic logic units, as well as control units, which make them very good at performing highly parallel operations simultaneously. Each SM computes a single block of the final matrix, and this operation is completed in parallel with many SMs. This increases the efficiency, since it utilises the local memory within each SM. Thus, the higher memory bandwidth of the GPU can be used to fetch the data from these blocks. This is a highly parallelised task, which allocates data into the GPU or the SM memory.


The operation is easy to define with numpy and PyTorch. [Source](https://www.kaggle.com/code/shadabhussain/cifar-10-cnn-using-pytorch?scriptVersionId=67771835&cellId=39).





In [3]:
import numpy as np
import torch

def f(A, K):
    """
    Input image, A, and kernel, K.
    """
    k, l = A.shape       # image dimensions
    m, n = K.shape      # kernel dimensions
    oi, oj = k-m+1, l-n+1  # output dimensions
    output = torch.zeros([oi, oj])
    for i in range(oi): 
        for j in range(oj):
            output[i,j] = torch.sum(image[i:i+m,j:j+n] * K)
    return output

We use the example from the following diagram, which we used in the project.

<img src="https://miro.medium.com/v2/resize:fit:1070/format:webp/1*Zx-ZMLKab7VOCQTxdZ1OAw.gif" alt="conv2d" style="width:300px;"/>


In [4]:
image = torch.tensor([
    [3, 3, 2, 1, 0], 
    [0, 0, 1, 3, 1], 
    [3, 1, 2, 2, 3], 
    [2, 0, 0, 2, 2], 
    [2, 0, 0, 0, 1]], dtype=torch.float32)

kernel = torch.tensor([
    [0, 1, 2], 
    [2, 2, 0], 
    [0, 1, 2]], dtype=torch.float32)

f(image, kernel)

tensor([[12., 12., 17.],
        [10., 17., 19.],
        [ 9.,  6., 14.]])

### <ins>Main challenges, and overcoming them:</ins>

1. A challenge I faced during this project was understanding how a convolutional neural network works. Many concepts of CNNs are subtle, and I try to explain them in the project. Of course, I would have loved to spend many weeks on this project, delving further into the nuances and subtleties of creating a successful high-accuracy image classification model, but there was only limited time: another challenge.

2. The timeframe to complete this project is very short. The brief requires us to look into various tricky concepts. The ideas and mathematics behind the workings and implementation of a CNN are complex. Additionally, the brief of the project also warranted a focus on parallel computing. However, to successfully explore this brief with image classification, one has to first understand a great many concepts used in CNNs, and deep learning. I tried my best to fit in as much explanation as possible within the timeframe, and I think that the layout of the project was successful.

3. Furthermore, the project requires implementing tricky concepts with code. Luckily, there are many resources online; however, the concepts and the code are complicated. I tried to provide clarification where possible.

4. Lack of GPU: My laptop does not have a dedicated GPU; therefore, I had to find a solution since most parallel computing utilises GPUs. It took some time, but I discovered that I could use the cloud GPUs in Google Colab. This was a vital resource; however, it proved to be financially expansive.


### <ins>Final remarks:</ins>

Various improvements could be made. These include adding more GPUs and using them in parallel to make more efficient the training of CNNs. Methods such as using 4 NVIDIA TITAN GPUs, each with 6GB of RAM, to implement a classification model $[1]$ on the ImageNet dataset $[2]$ have been studied.

Additionally, there are many more methods of optimisation that there was simply not enough time to explore in great detail. More layers could be used in the network to provide more depth. 

Assume that the number of data points was to be increased by a factor of 1000. This might necessitate the use of a distributed database management system (DBMS). In such a system, the data is stored across various different physical locations. These could be various different computers in one location, such as a data centre, or a network of dispersed interconnected devices. These systems share no physical components, and they are more tolerant to faults, since there are tools to find and isolate any failures that arise and automatically fix them. With large datasets, they are faster and more efficient in machine learning since they allow for parallel computation by using multiple different computing resources simultaneously. This means they can operate with data on much larger scales than single machines. For large-scale data operations, this can result in significant financial savings.

The CIFAR-10 dataset is slightly idealised, although the images are from a real-world setting. The images are of relatively low-resolution. In the real world, we are likely to run into data that may require image imputation: the complex task of creating higher-quality images from lower-quality ones, or images with missing data values. This was briefly noted in the project, and we referenced the implementation of this task in analysing medical images. Furthermore, in the real world, image data often has a far larger number of pixels, as well as overlaps in classes. This would require a more sophisticated neural network model. Such neural networks $[3]$ have been used on the more diverse ImageNet dataset.

Ultimately, I think the project was broadly successful. Various theoretical graphs were replicated from the literature using experimental analysis. These were clear, and illustrative, and provided elucidation to the functioning of the code, and the results output. We considered the ideas of GPU parallelism within the context of image classification and convolutional neural networks, and the results we presented helped to answer the question we posed at the start of the project. A fairly sophistical neural network was implemented at the end, and trained on the CIFAR-10 dataset to give a validation accuracy of over 80%.

## <ins>References</ins>

$[1]$ Yadan,  Omry and Adams,  Keith, et al. (2013). Multi-GPU Training of ConvNets.

$[2]$ Deng, Jia and Dong, Wei, et al. (2009). ImageNet: A large-scale hierarchical image database. 2009 IEEE Conference on Computer Vision and Pattern Recognition.

$[2]$ Krizhevsky,  Alex and Sutskever,  Ilya and Hinton,  Geoffrey E. (2017). ImageNet classification with deep convolutional neural networks. Communications of the ACM.
